In [1]:
import csv
import uuid
import datetime
from string import Template

In [2]:
inputCSVfile = '../output/predictions.csv'
outputTrigFile = '../output/classifications.trig'
outputGraph = 'http://resource.swissartresearch.net/graph/classifications'
dateTime = datetime.datetime.now()
addTimestampToGraph = True

In [3]:
if addTimestampToGraph:
    outputGraph = outputGraph + '-' + dateTime.strftime("%Y-%m-%d-%H-%M-%S")

In [4]:
data = []
with open(inputCSVfile, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        data.append(row)

In [5]:
# Debug
#data = data[:10]

In [6]:
namespaces = """
@prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
"""


In [7]:
output = namespaces

In [8]:
typeTemplate = Template("""
    <$graph> {
        <$assignedType> a crm:E55_Type ;
            rdfs:label "$label" .
    }
""")
for prediction in set([d['prediction'] for d in data]):
    typeUri = 'https://resource.swissartresearch.net/type/classification/' + prediction.replace(' ','')
    output += typeTemplate.substitute(graph=outputGraph,
                                     assignedType=typeUri,
                                     label=prediction)

In [9]:
technique = 'https://github.com/swiss-art-research-net/bso-image-classification/blob/583c8891df9739723f81dfc1ac127b06e411ec34/models/model.pkl'

classificationTemplate = Template("""
    <$graph> {
    
        <$classification> a crm:E13_Attribute_Assignment ;
            crm:P140_assigned_attribute_to <$subject> ;
            crm:P141_assigned <$assignedType> ;
            crm:P4_has_time-span <$classification/date> ;
            crm:P33_used_specific_technique <$technique> ;
            rdf:value "$confidence"^^xsd:float .
            
        <$classification/date> a crm:E52_Time-Span ;
            crm:P82_at_some_time_within "$dateTime"^^xsd:dateTime .
    }
""")

for row in data:
    classificationUri = 'https://resource.swissartresearch.net/classification/' + str(uuid.uuid4())
    subjectUri = 'https://resource.swissartresearch.net/artwork/' + row['id']
    typeUri = 'https://resource.swissartresearch.net/type/classification/' + row['prediction'].replace(' ','')
    output += classificationTemplate.substitute(graph=outputGraph, 
                                  classification=classificationUri, 
                                  subject=subjectUri, 
                                  assignedType=typeUri,
                                  technique=technique,
                                  confidence=row['confidence'],
                                  dateTime=dateTime.strftime("%Y-%m-%dT%H:%M:%S"))

In [10]:
with open(outputTrigFile, 'w') as f:
    f.write(output)